In [ ]:
# ================================
# Mental Fatigue Detection Model
# Training on ESP32 dataset
# ================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -------------------------------
# 1. Load Dataset
# -------------------------------
# CSV format: tilt, light, label
# tilt = float (degrees), light = 0/1, label = 0=Normal, 1=Fatigue

data = pd.read_csv("dataset.csv")

X = data[["tilt", "light"]].values
y = data["label"].values

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -------------------------------
# 2. Define Neural Network
# -------------------------------
model = keras.Sequential([
    layers.Input(shape=(2,)),
    layers.Dense(8, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# -------------------------------
# 3. Train Model
# -------------------------------
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=16,
    validation_split=0.2,
    verbose=0
)

# -------------------------------
# 4. Evaluate Model
# -------------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.2f}")
print(classification_report(y_test, (model.predict(X_test) > 0.5).astype(int)))

# -------------------------------
# 5. Export TensorFlow Lite Model
# -------------------------------
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("mental_fatigue_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Exported model: mental_fatigue_model.tflite")
